# Trios Compilation Example

### Imports

In [37]:
import sys
import numpy as np
from qiskit import *
from qiskit.compiler import transpile
from qiskit.test.mock import FakeMumbai

sys.path.append('..')
from trios import trios_toffoli_transpile, trios_transpile

### Load Demo Backend

In [38]:
IBMQ.load_account()
provider = IBMQ.providers()[1]
backend = FakeMumbai()

ibmqfactory.load_account:WARNING:2022-03-12 13:53:57,758: Credentials are already in use. The existing account in the session will be replaced.


### Build Test Circuit

In [39]:
#test circuit
test_circuit = QuantumCircuit(7, 7)
test_circuit.h(0)
test_circuit.cx(0, 1)
test_circuit.ccx(1, 2, 3)
test_circuit.x(2)
test_circuit.ccx(2, 3, 4)
test_circuit.h(3)
test_circuit.cx(3, 2)
test_circuit.h(4)
test_circuit.ccx(4, 3, 1)
test_circuit.ccx(3, 4, 5)
test_circuit.ccx(4, 5, 6)

test_circuit.measure(list(range(7)), list(range(7)))
print(test_circuit)

     ┌───┐          ┌─┐                                                     
q_0: ┤ H ├──■───────┤M├─────────────────────────────────────────────────────
     └───┘┌─┴─┐     └╥┘                    ┌───┐        ┌─┐                 
q_1: ─────┤ X ├──■───╫─────────────────────┤ X ├────────┤M├─────────────────
          └───┘  │   ║ ┌───┐          ┌───┐└─┬─┘┌─┐     └╥┘                 
q_2: ────────────■───╫─┤ X ├──■───────┤ X ├──┼──┤M├──────╫──────────────────
               ┌─┴─┐ ║ └───┘  │  ┌───┐└─┬─┘  │  └╥┘      ║      ┌─┐         
q_3: ──────────┤ X ├─╫────────■──┤ H ├──■────■───╫───■───╫──────┤M├─────────
               └───┘ ║      ┌─┴─┐├───┤       │   ║   │   ║      └╥┘┌─┐      
q_4: ────────────────╫──────┤ X ├┤ H ├───────■───╫───■───╫───■───╫─┤M├──────
                     ║      └───┘└───┘           ║ ┌─┴─┐ ║   │   ║ └╥┘┌─┐   
q_5: ────────────────╫───────────────────────────╫─┤ X ├─╫───■───╫──╫─┤M├───
                     ║                           ║ └───┘ ║ ┌─┴─┐ ║  ║ └╥┘┌─┐

### Run Trios Compiler

In [40]:
trios_compiled_circuit = trios_toffoli_transpile(circs=[test_circuit], backend=backend, basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'ccx', 'ccx_linear', 'reset'], seed_transpiler=0)[0]

layout_method:  dense
routing method:  basic
ccx
ccx
ccx
ccx
ccx
No solution found:  nonexistent solution
Layout({
5: Qubit(QuantumRegister(7, 'q'), 0),
7: Qubit(QuantumRegister(7, 'q'), 1),
3: Qubit(QuantumRegister(7, 'q'), 2),
4: Qubit(QuantumRegister(7, 'q'), 3),
2: Qubit(QuantumRegister(7, 'q'), 4),
1: Qubit(QuantumRegister(7, 'q'), 5),
0: Qubit(QuantumRegister(7, 'q'), 6)
})
The arguments for the toffoli node are:  Qubit(QuantumRegister(27, 'q'), 4) Qubit(QuantumRegister(27, 'q'), 2) Qubit(QuantumRegister(27, 'q'), 1)
The distances between the toffoli qubits are:  2 between qubits 0 and 1
The distances between the toffoli qubits are:  1 between qubits 1 and 2
The distances between the toffoli qubits are:  1 between qubits 0 and 2
The physical qubits for the toffoli are:  4 2 1
The required toffoli will be decomposed using an 8 cnot decomposition - two in center
The arguments for the toffoli node are:  Qubit(QuantumRegister(27, 'q'), 2) Qubit(QuantumRegister(27, 'q'), 1) Qubit(Quan

### Compare Trios Results to IBM Results

In [41]:
ibm_compiled_circuit = transpile(test_circuit, backend=backend, basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'reset'], optimization_level=3, seed_transpiler=0)

In [42]:
print("              Trios |  IBM")
print("==============================")
print(f"Circuit size:  {trios_compiled_circuit.size()}   |   {ibm_compiled_circuit.size()}")
print(f"Circuit depth: {trios_compiled_circuit.depth()}   |   {ibm_compiled_circuit.depth()}")

              Trios |  IBM
Circuit size:  69   |   164
Circuit depth: 45   |   104


In [43]:
trios_compiled_circuit.count_ops()

OrderedDict([('cx', 33),
             ('rz', 15),
             ('sx', 8),
             ('measure', 7),
             ('ccx_linear', 5),
             ('x', 1),
             ('barrier', 1)])

In [44]:
ibm_compiled_circuit.count_ops()

OrderedDict([('cx', 75),
             ('rz', 60),
             ('sx', 20),
             ('measure', 7),
             ('x', 2),
             ('barrier', 1)])